<a href="https://colab.research.google.com/github/chinmay5/guided-research/blob/colab/Guided_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing all the libraries needed for the task
import pandas as pd
import json
import nltk
from pandas.io.json import json_normalize
nltk.download('stopwords')
nltk.download('punkt')
import re  
from nltk.corpus import stopwords
stops1 = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
import ipywidgets
import traitlets

In [0]:
processed_df = pd.read_pickle('./processed_df.pkl')

In [0]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(processed_df, test_size = 0.2, random_state = 0)

## Neural Network Part

In [0]:
# I am assuming that we are able to get the categories here
# Now this should become a LSTM based model which will try and do binary prediction
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.utils.data.dataloader as dataloader
from torch.nn.utils.rnn import pad_sequence

In [0]:
# Some constants
batch_size = 1000

In [0]:
# https://github.com/yunjey/seq2seq-dataloader/blob/master/data_loader.py
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (src_seq, trg_seq).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of tuple (src_seq, trg_seq).
            - src_seq: torch tensor of shape (?); variable length.
            - trg_seq: torch tensor of shape (?); variable length.
    Returns:
        src_seqs: torch tensor of shape (batch_size, padded_length).
        src_lengths: list of length (batch_size); valid length for each padded source sequence.
        trg_seqs: torch tensor of shape (batch_size, padded_length).
        trg_lengths: list of length (batch_size); valid length for each padded target sequence.
    """
    def merge(sequences):
        lengths = [len(seq) for seq in sequences]
        padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        for i, seq in enumerate(sequences):
            end = lengths[i]
            padded_seqs[i, :end] = seq[:end]
        return padded_seqs, lengths

    # sort a list by sequence length (descending order) to use pack_padded_sequence
#     print(data[0]) # list of tuples
    data.sort(key=lambda x: len(x[0]), reverse=True)

    # seperate source and target sequences
    src_seqs, trg_seqs = zip(*data)

    # merge sequences (from tuple of 1D tensor to 2D tensor)
    src_seqs, src_lengths = merge(src_seqs)
    # target sequence for us is a single tensor so we do not need to 
    # merge it
    #trg_seqs, trg_lengths = merge(trg_seqs)
    trg_seqs = torch.as_tensor(trg_seqs)
    return src_seqs, src_lengths, trg_seqs #, trg_lengths


In [0]:
class RecipeData(data.Dataset):
    
    def __init__(self, df):
        super(RecipeData, self).__init__()
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        X = torch.as_tensor(self.df.Ingredient_Numeric.iloc[idx])
        y = torch.as_tensor(self.df.Recipe_id_numeric.iloc[idx])
        return X,y
    

In [0]:
train_dataset = RecipeData(df_train)
test_dataset = RecipeData(df_test)
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                               drop_last=True) # Done for cases when num_samples not exact multiple
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, # of the batch_size
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                              drop_last=True)

In [10]:
df_train.Recipe_id_numeric[4]

5

In [11]:
train_dataset[0]

(tensor([ 65,  33, 140,  36]), tensor(0))

In [12]:
for (X,X_len,y) in train_data_loader:
    print(X)
    break

tensor([[129,  70,  36,  ...,  24,  89, 243],
        [ 24,  42,  49,  ...,   0,   0,   0],
        [ 39,  55, 243,  ...,   0,   0,   0],
        ...,
        [ 27,   0,   0,  ...,   0,   0,   0],
        [131,   0,   0,  ...,   0,   0,   0],
        [221,   0,   0,  ...,   0,   0,   0]])


In [13]:
# Moving things to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [0]:
class RecipePredictor(nn.Module):
    
    def __init__(self, vocab_size, hidden_dim, embedding_dim,
                 batch_size, output_dim, device):
        
        super(RecipePredictor, self).__init__()
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.hidden_dim)
        self.batch_size = batch_size
        self.predictor = nn.Linear(self.hidden_dim, self.output_dim)
        self.non_linearity = nn.ReLU()
        self.init_hidden() # TODO:  This should happen at the beginning of each epoch
        
    def init_hidden(self):
        self.h_n = torch.randn((1, self.batch_size, self.hidden_dim) ,device=device)
        self.c_n = torch.randn((1, self.batch_size, self.hidden_dim) , device=device)
        
        
    
    def forward(self, input_sequence, max_len):
#         print(input_sequence)
#         print(max_len)
        embedded = self.embedding(input_sequence)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, max_len, batch_first=True)
        outputs, (self.c_n, self.h_n) = self.lstm(packed, (self.c_n, self.h_n))
        # Unpack padding
        """
            Honestly, I do not know if at this point, I need the output. I would rather prefer to work with the
            self.h_n cell and so will not `pad_padded_sequence`
        """
        #outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        # on which output should the prediction be done?
        # self.h_n =  num_layers, batch_size, hidden_dim
        batch_size = self.h_n.shape[1]
        output_predicted = self.predictor(self.h_n.reshape(batch_size, -1))
        return output_predicted
        

In [0]:
# These are some constants that I am just copying here
# These need to be updated based on the files
ingredient_vocabulary = 1152
recipe_vocabulary = 16
vocab_size = ingredient_vocabulary + 1

In [0]:
num_recipes = recipe_vocabulary + 1
model = RecipePredictor(vocab_size=vocab_size, hidden_dim=512, embedding_dim=300, batch_size=batch_size,
                       output_dim=num_recipes, device=device)

### I did not put hidden state on cuda and that was an issue. So, it has been corrected

In [17]:
model.to(device)

RecipePredictor(
  (embedding): Embedding(1153, 300)
  (lstm): LSTM(300, 512)
  (predictor): Linear(in_features=512, out_features=17, bias=True)
  (non_linearity): ReLU()
)

In [18]:
model(X.cuda(), torch.tensor(X_len).cuda())

tensor([[ 0.2426,  0.2468,  0.1269,  ...,  0.2474,  0.0481, -0.1306],
        [-0.1433,  0.0131,  0.2130,  ..., -0.0695, -0.0562, -0.3213],
        [ 0.1079,  0.0507,  0.0994,  ..., -0.2803, -0.1576,  0.1823],
        ...,
        [-0.0101,  0.0532,  0.4223,  ...,  0.3989,  0.0642, -0.2718],
        [-0.1152,  0.3523, -0.3859,  ..., -0.2631, -0.5250,  0.0990],
        [-0.1229,  0.1882,  0.2236,  ...,  0.0140,  0.2446, -0.3033]],
       device='cuda:0', grad_fn=<AddmmBackward>)

## Put loss function and categories

In [0]:
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [0]:
num_epoch = 20

In [0]:
def put_elements_to_device(a,b,c,device):
    return a.to(device), b.to(device), c.to(device)

In [0]:
!pip install -q tb-nightly

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [24]:
print(len(train_data_loader) * batch_size)
print(len(df_train))
# Not same since we are dropping some terms which do not match up
N_train = len(train_data_loader) * batch_size
N_test = len(test_data_loader) * batch_size

79000
79948


In [0]:
for epoch in range(num_epoch):
    running_loss = 0
    model.init_hidden()
    for idx,(X,X_len,y) in enumerate(train_data_loader):
        model.zero_grad()
        X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
        prediction = model(X, X_len)
        loss = criterion(prediction, y)
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if idx % 100 == 0:
            print("epoch {} loss {}".format(epoch, running_loss))
    writer.add_scalar('Loss/train', running_loss/N_train, epoch)
    # validation set is still left to create
    # Now to test the validation set
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for idx,(X,X_len,y) in enumerate(test_data_loader):
            X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
            outputs = model(X,X_len)
            loss = criterion(outputs, y)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += X.size(0)
            correct += (predicted == y).sum().item()
    writer.add_scalar('Loss/train', test_loss/N_test, epoch)
    print('Accuracy of the network on the test samples: %d %%' % (
        100 * correct / total))
        
            

epoch 0 loss 2.8605995178222656


In [0]:
# !pip install tensorboard
%load_ext tensorboard
%tensorboard --logdir runs

In [0]:
# !pip uninstall tensorboard
# !pip install --force-reinstall tf-nightly-2.0-preview